<a href="https://github.com/data-for-good-grenoble/atelier-OFF" target="_blank"><img src="image_meetup_off.png" width=500px/></a>

# 🍫 Atelier d'exploration des données d'Open Food Facts - Meetup Python Grenoble 🐍

*Jeudi 27 juin 2024 - 19h - <a href="https://turbine.coop/" target="_blank">La Turbine</a> - Grenoble*

**Notations :**

▶️ : le code peut être exécuté sans modification

💻 : le code doit être créé ou modifié

___

▶️ Importer les **bibliothèques nécessaires** à l'activité

In [ ]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

## 🎣 PARTIE N°1 : Récupérer un produit de la base de données d'Open Food Facts et classer les produits par similitudes

Le modèle de recommandation que l'on construit se base sur des **similitudes des caractéristiques des produits** puis en sélectionnant les produits avec le **meilleur nutriscore**. D'autres **approches par catégories de produits** sont possibles. 

▶️ Récupérer **500 produits** de la **catégorie "snacks"**

In [ ]:
def get_open_food_facts_data(category, page_size=100, num_pages=5):
    base_url = "https://world.openfoodfacts.org/cgi/search.pl"
    products = []
    for page in range(1, num_pages + 1):
        params = {
            'action': 'process',
            'tagtype_0': 'categories',
            'tag_contains_0': 'contains',
            'tag_0': category,
            'page_size': page_size,
            'page': page,
            'json': 1
        }
        response = requests.get(base_url, params=params)
        data = response.json()
        products.extend(data.get('products', []))
    return products

category = 'snacks'
data = get_open_food_facts_data(category)
df = pd.json_normalize(data)
df

▶️ Analyser les **valeurs manquantes**

In [ ]:
sns.heatmap(df.isna(), cbar=False)
plt.show()

▶️ Sélectionner les **colonnes les plus intéressantes** (5 à 10 colonnes)

In [ ]:
# Sélectionner les colonnes d'intérêt
columns = ['product_name', 'nutriments.energy_100g', 'nutriments.fat_100g',
           'nutriments.carbohydrates_100g', 'nutriments.proteins_100g', 'nutriments.sugars_100g',
           'nutriments.salt_100g']
df = df[columns]
# Renommer les colonnes pour simplifier
df.columns = ['product_name', 'energy', 'fat', 'carbohydrates', 'proteins', 'sugars', 'salt']

▶️ Traiter les **valeurs manquantes**

In [ ]:
df = df.dropna()

In [ ]:
sns.heatmap(df.isna(), cbar=False)
plt.show()

▶️ **Normaliser** les données

In [ ]:
# Normaliser les données
nutritional_data = df[['energy', 'fat', 'carbohydrates', 'proteins', 'sugars', 'salt']]
normalized_data = (nutritional_data - nutritional_data.mean()) / nutritional_data.std()

▶️ Calculer la **matrice de similarité cosinus**

In [ ]:
# Calculer la similarité cosinus
similarity_matrix = cosine_similarity(normalized_data)

▶️ Créer le **moteur de recommandation** de produits similaires

In [ ]:
def recommend_products(product_name, df, similarity_matrix, top_n=5):
    matching_products = df[df['product_name'].str.contains(product_name, case=False, na=False)]
    if matching_products.empty:
        print(f"Produit '{product_name}' non trouvé.")
        return pd.DataFrame()
    product_index = matching_products.index[0]
    similarity_scores = list(enumerate(similarity_matrix[product_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similar_products_indices = [i[0] for i in similarity_scores[1:top_n + 1]]
    return df.iloc[similar_products_indices][['product_name', 'energy', 'fat', 'carbohydrates', 'proteins', 'sugars', 'salt']]

▶️ Tester le moteur de recommandation sur un **exemple de produit**

In [ ]:
# Exemple de recommandation
product_name = "Crackers"
recommended_products = recommend_products(product_name, df, similarity_matrix, top_n=5)
print(recommended_products)

## 🌍 PARTIE N°2 : trouver des produits avec un meilleur impact environnmental

Les données de la base de données d'Open Food Facts ont été **sélectionnées** suivant les critères suivants :

- Produits présents en **France**
- Avec un **nutriscore valide**
- Avec un **écoscore valide**
- Avec des colonnes avec **moins de 20% de données manquantes**
- Avec une sélection de **colonnes intéressantes**
- Avec une sélection aléatoire de **10% des données** (problèmes de temps de calculs)

___

▶️ **Récupérer les données** pré-traitées de la base de données d'Open Food Facts

In [ ]:
data = pd.read_csv("OFF_dataset.csv")
data

▶️ **Remplacer les valeurs manquantes** de la colonne "brands" par "-". Vérifier le **remplissage**

In [ ]:
data['brands'] = data['brands'].fillna("-")

In [ ]:
sns.heatmap(data.isna(), cbar=False)
plt.show()

▶️ **Supprimer les valeurs manquantes restantes** et vérifier qu'il ne reste plus de valeurs manquantes

In [ ]:
data = data.dropna()

In [ ]:
sns.heatmap(data.isna(), cbar=False)
plt.show()

💻 **Sélectionner les données** à prendre en compte et **normaliser ces données**

In [ ]:
# A COMPLETER

▶️ Calculer la **matrice de similarité cosinus**

In [ ]:
# Calculer la similarité cosinus
similarity_matrix = cosine_similarity(normalized_data)

💻 **Modifier le moteur de recommandation** de produits similaires et **tester le fonctionnement** sur différents produits

In [ ]:
# A COMPLETER